In [10]:
import numpy as np
import pandas as pd
import urllib
from requests.exceptions import HTTPError
import bs4
import selenium
from selenium import webdriver
import time

In [2]:
links = ['20190428', '20190331', '20190303', '20190203', '20190106', '20181209', '20181111', '20181014', '20180916', '20180819']

In [3]:
def fetch_website(links):
    urls = []
    url = 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/'
    urls.append(url)
    url += 'sp'
    for i, l in  enumerate(links):
        url += '-d-' + links[i]
        urls.append(url)
    return urls

In [4]:
urls = fetch_website(links)
urls

['https://www.msn.com/en-us/sports/soccer/la-liga/scores/',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106-d-20181209',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106-d-20181209-d-20181111',
 'https://www.msn.com/en-us/sports/soccer/la-liga/scores/sp-d-20190428-d-20190331-d-20190303-d-20190203-d-20190106-d-20181209-d-20181111-d-20181014',
 'https://www.msn.com/en-us/sports/soccer/la-lig

In [22]:
def get_stats(stats):
    stat_list = ['BALL', 'SHOTS', 'FOULS', 'CORNERS', 'SAVES', 'OFFSIDES', 'YELLOW', 'RED']
    len_stats = len(stats)
    home_shots_og = None
    away_shots_og = None
    home_saves = None
    away_saves = None
    home_offsides = None
    away_offsides = None
    home_poss = None
    away_poss = None
    home_fouls = None
    away_fouls = None
    home_shots = None
    away_shots = None
    home_corners = None
    away_corners = None
    home_yellow = None
    away_yellow = None
    home_red = None
    away_red = None
    for i in range(len_stats):
        if stats[i][1] in stat_list:
            if stats[i][1] == 'BALL':
                home_poss = stats[i][0]
                away_poss = stats[i][-1]
            if stats[i][1] == 'FOULS':
                home_fouls = stats[i][0]
                away_fouls = stats[i][-1]
            if stats[i][1] == 'CORNERS':
                home_corners = stats[i][0]
                away_corners = stats[i][-1]
            if stats[i][1] == 'SAVES':
                home_saves = stats[i][0]
                away_saves = stats[i][-1]
            if stats[i][1] == 'OFFSIDES':
                home_offsides = stats[i][0]
                away_offsides = stats[i][-1]
            if stats[i][1] == 'YELLOW':
                home_yellow = stats[i][0]
                away_yellow = stats[i][-1]
            if stats[i][1] == 'RED':
                home_red = stats[i][0]
                away_red = stats[i][-1]
            if stats[i][1] == 'SHOTS':
                if len(stats[i]) == 3:
                    home_shots = stats[i][0]
                    away_shots = stats[i][-1]
                else:
                    home_shots_og = stats[i][0]
                    away_shots_og = stats[i][-1]
    return(home_poss, away_poss, home_shots, away_shots, home_shots_og, away_shots_og, home_fouls, away_fouls, home_corners, 
           away_corners, home_saves, away_saves, home_offsides, away_offsides, home_yellow, away_yellow, home_red, away_red)

In [25]:
teams = []
html = urllib.request.urlopen('https://www.msn.com/en-us/sports/soccer/la-liga/scores/').read()
raw = bs4.BeautifulSoup(html, 'lxml')
results = raw.find('div', {'class': 'sectionsgroup'}).findAll('tbody', {'class': ['even rowlink', 'odd rowlink']})
for r in results:
    home_team = r.find('td', {'class': 'teamname teaminline alignright size23'}).text.strip()
    away_team = r.find('td', {'class': 'teamname size23'}).text.strip()
    data_link = 'https://www.msn.com' + r.find('a').get('href')
    try:
        html_link = urllib.request.urlopen(data_link)
        raw_link = bs4.BeautifulSoup(html_link, 'lxml')
        result_link = raw_link.find('div', {'id': 'main'}).find('div', {'id': 'matchupgamestatsmodule'}).findAll('div', {'class': 'statsitem'})
    except urllib.error.HTTPError:
        html_link = urllib.request.urlopen('https://www.msn.com/en-us/sports/soccer/la-liga/eibar-v-barcelona/game-center/sp-id-80402000001009693')
        raw_link = bs4.BeautifulSoup(html_link, 'lxml')
        result_link = raw_link.find('div', {'id': 'main'}).find('div', {'id': 'matchupgamestatsmodule'}).findAll('div', {'class': 'statsitem'})
    stats = []
    for rl in result_link:
        stats.append(rl.text.split())
    res = get_stats(stats)
    teams.append([home_team, away_team, data_link, res[0], res[1]])
    time.sleep(5)

In [26]:
teams

[['Eibar',
  'Barça',
  'https://www.msn.com/en-us/sports/soccer/la-liga/eibar-v-barcelona/game-center/sp-id-80402000001009693',
  '39',
  '61'],
 ['Real Madrid',
  'Betis',
  'https://www.msn.com/en-us/sports/soccer/la-liga/real-madrid-v-real-betis/game-center/sp-id-80402000001009694',
  '47',
  '53'],
 ['Alavés',
  'Girona',
  'https://www.msn.com/en-us/sports/soccer/la-liga/alav%c3%a9s-v-girona/game-center/sp-id-80402000001009686',
  '43',
  '57'],
 ['Celta',
  'Rayo',
  'https://www.msn.com/en-us/sports/soccer/la-liga/celta-de-vigo-v-rayo-vallecano/game-center/sp-id-80402000001009687',
  '45',
  '55'],
 ['Huesca',
  'Leganés',
  'https://www.msn.com/en-us/sports/soccer/la-liga/huesca-v-legan%c3%a9s/game-center/sp-id-80402000001009690',
  '50',
  '50'],
 ['Espanyol',
  'Real Sociedad',
  'https://www.msn.com/en-us/sports/soccer/la-liga/espanyol-v-real-sociedad/game-center/sp-id-80402000001009688',
  '37',
  '63'],
 ['Getafe',
  'Villarreal',
  'https://www.msn.com/en-us/sports/socce